Data Cleaning

----

Imports

In [1]:
import pandas as pd

# this identifies emojis
import emoji

import re

# this setting widens how many characters pandas will display in a column:
pd.options.display.max_colwidth = 400


----

#### Load Data

In [2]:
# Data pulled from January 2023
bg_df_07 = pd.read_csv("../data/api_pull/1_7_2023_boardgames.csv")
wsb_df_07 = pd.read_csv("../data/api_pull/1_7_2023_wsb.csv")

bg_df_08 = pd.read_csv("../data/api_pull/1_8_2023_boardgames.csv")
wsb_df_08 = pd.read_csv("../data/api_pull/1_8_2023_wsb.csv")

bg_df_10 = pd.read_csv("../data/api_pull/1_10_2023_boardgames.csv")
wsb_df_10 = pd.read_csv("../data/api_pull/1_10_2023_wsb.csv")

bg_df_13 = pd.read_csv("../data/api_pull/1_13_2023_boardgames.csv")
wsb_df_13 = pd.read_csv("../data/api_pull/1_13_2023_wsb.csv")


#### Add `subreddit` column 

In [3]:
df_list = []
bg_list = [
    bg_df_07,
    bg_df_08,
    bg_df_10,
    bg_df_13,
]
wsb_list = [wsb_df_07, wsb_df_08, wsb_df_10, wsb_df_13]


def add_col(df, val):
    df["subreddit"] = val


for bg in bg_list:
    add_col(bg, "boardgames")
    df_list.append(bg)

for wsb in wsb_list:
    add_col(wsb, "wsb")
    df_list.append(wsb)


#### Combine All DataFrames

In [4]:
df = pd.concat(df_list)
df.shape

(12926, 10)

In [6]:
df["subreddit"].unique()


array(['boardgames', 'wsb'], dtype=object)

In [7]:
df.head(1)

,selftext,title,score,domain,id,author,num_comments,url,created_utc,subreddit
0,"If I wanted to play a virtual board game remotely with someone who didn't have a great PC but could still Zoom/Skype, what would you recommend? Any direction you can provide would be massively appreciated.\n\n&amp;#x200B;\n\nHere's the scenario I'd like to discover a solution for:\n\n* A roleplay game I can essentially DM on one side - She says ""Let's move here"" and I can adjust the tokens on ...",Virtual board games you can host over Zoom?,1,self.boardgames,105yauz,TylerDeLarm,0,https://www.reddit.com/r/boardgames/comments/105yauz/virtual_board_games_you_can_host_over_zoom/,1673122098,boardgames


----


- Cleanup `na`

In [8]:
df.isna().sum()


selftext        5912
title              0
score              0
domain            31
id                 0
author             0
num_comments       0
url               31
created_utc        0
subreddit          0
dtype: int64

In [9]:
# fill na in selftext with empty string
df["selftext"] = df["selftext"].fillna("")


In [10]:
df.isna().sum()

selftext         0
title            0
score            0
domain          31
id               0
author           0
num_comments     0
url             31
created_utc      0
subreddit        0
dtype: int64

In [11]:
df_missing_ratio = round(df.isna().sum().max() / df.value_counts().sum(), 3)

print(
    f"""
Missing Ratio: {df_missing_ratio}%
"""
)



Missing Ratio: 0.002%



In [12]:
# Drop na since it is only fraction of dataframes
df.dropna(inplace=True)
df.isna().sum()


selftext        0
title           0
score           0
domain          0
id              0
author          0
num_comments    0
url             0
created_utc     0
subreddit       0
dtype: int64

----
- Check if `selftext` contains `nan` as text

In [13]:
df_st_nan_ratio = (
    df[df["selftext"] == "nan"].value_counts().sum() / df.value_counts().sum()
)

print(
    f"""
DF-selftext Missing Ratio which contains nan as string: {df_st_nan_ratio}%
"""
)



DF-selftext Missing Ratio which contains nan as string: 0.0%



In [14]:
# Since selftext field contain nan as string
# it was missed during dropna. Will transform nan
# to empty string
df["selftext"] = df["selftext"].map(lambda text: "" if text == "nan" else text)


----
- Display `dtypes`

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12895 entries, 0 to 999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   selftext      12895 non-null  object
 1   title         12895 non-null  object
 2   score         12895 non-null  int64 
 3   domain        12895 non-null  object
 4   id            12895 non-null  object
 5   author        12895 non-null  object
 6   num_comments  12895 non-null  int64 
 7   url           12895 non-null  object
 8   created_utc   12895 non-null  int64 
 9   subreddit     12895 non-null  object
dtypes: int64(3), object(7)
memory usage: 1.1+ MB


----
- Strip `selftext` and `title` from extra spaces and `\n`

In [16]:
def strip_column(df, target_list):
    for target in target_list:
        df[target] = df[target].map(lambda text: str(text).strip())


In [17]:
targets = ["selftext", "title"]
strip_column(df, targets)


In [18]:
df["selftext"] = df["selftext"].map(lambda text: re.sub("\n", "", text))


----
- Create `selftext` and `title` length columns

In [19]:
def add_length_column(df, targets):
    for target in targets:
        new_col_name = target + "_length"
        df[new_col_name] = df[target].apply(len)


In [20]:
add_length_column(df, ["selftext", "title"])


In [21]:
df[["selftext_length", "title_length"]].sort_values(
    by="selftext_length", ascending=False
).tail(1)


,selftext_length,title_length
3604,0,91


----
- Create `selftext` and `title` word count columns

In [22]:
def add_word_count_column(df, targets):
    for target in targets:
        new_col_name = target + "_word_count"
        df[new_col_name] = df[target].map(lambda x: len(str(x).split(" ")))


In [23]:
add_word_count_column(df, ["title", "selftext"])


In [24]:
df[["selftext_word_count", "title_word_count"]].head(1)


,selftext_word_count,title_word_count
0,112,8


----
- Check `dtypes`

In [25]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 12895 entries, 0 to 999
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   selftext             12895 non-null  object
 1   title                12895 non-null  object
 2   score                12895 non-null  int64 
 3   domain               12895 non-null  object
 4   id                   12895 non-null  object
 5   author               12895 non-null  object
 6   num_comments         12895 non-null  int64 
 7   url                  12895 non-null  object
 8   created_utc          12895 non-null  int64 
 9   subreddit            12895 non-null  object
 10  selftext_length      12895 non-null  int64 
 11  title_length         12895 non-null  int64 
 12  title_word_count     12895 non-null  int64 
 13  selftext_word_count  12895 non-null  int64 
dtypes: int64(7), object(7)
memory usage: 1.5+ MB


----
- Count `emojis` used in `selftext` and `title`

In [26]:
def add_emoji_count_column(df, target_list):
    for target in target_list:
        new_target = target + "_emoji_count"
        df[new_target] = df[target].map(
            lambda text: len([word for word in text if emoji.distinct_emoji_list(word)])
        )


In [27]:
add_emoji_count_column(df, ["selftext", "title"])


In [28]:
df[["selftext_emoji_count"]].sort_values(
    by="selftext_emoji_count", ascending=True
).tail(5)


,selftext_emoji_count
2392,76
3822,96
2885,133
4283,144
3340,153


----
- Output the data

In [29]:
df.to_csv("../data/cleaned_data/reddit_data.csv", index=False)
